In [4]:
import json
import os
import sys
import numpy as np
import time
import urllib.request
import traceback
from transformers import LlamaForCausalLM, LlamaTokenizer
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import BSHTMLLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from bs4 import BeautifulSoup
import requests
import lxml
import logging

import src.analyse as al
import src.queries as q

In [5]:
no_factors= 5
no_links = 5
analysis_id = 'nymex_crude_oil'
subject = 'nymex crude oil'
model_props = {
        "name": "ggml-model-q4_0.gguf",
        "n_gpu_layers":40,
        "n_batch":1024,
        "n_ctx":4096
    }

In [ ]:
def LoadModel(model_props, grammar):
    model_path = os.path.join(os.getcwd(), 'models', model_props['name'])
    grammar_path = os.path.join(os.getcwd(), 'models', grammar)
    llm = LlamaCpp(model_path=model_path, 
        temperature=0.0, 
        top_p=1,
        n_ctx=model_props['n_ctx'], 
        seed = 42,
        verbose=True, 
        n_gpu_layers=model_props['n_gpu_layers'],
        n_batch=model_props['n_batch'],
        #grammar_path=grammar_path
    )
    return llm

#llm = LoadModel(model_props, 'json.gbnf')

In [5]:
question = """
The opposite of "supply cuts by Russia" is "
"""
llm(question)

Llama.generate: prefix-match hit


'\nAnswer: Demand increases by the West.'

In [6]:
str(query.__name__)

'Effect_Q0'

In [41]:
def BingToActualURL(url):
    headers = {
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582" 
        }
    response = requests.get(url, headers = headers).text
    start_ind = response.index('var u = "')
    if start_ind >= 0:
        response = response[start_ind+9:]
        end_ind = response.find('";')
        if end_ind >=0:
            response = response[:end_ind]
            return response
    return None

def SearchBingGetLinksAndSave(queries, no_links, config_file, analysis_id, doc_prefix):
    try:
        doc_id = 0
        for query in queries:
            logging.info('running bing search for query: ' + query)
            url = "https://www.bing.com/search?form=QBRE&q="+query.replace(' ', '+')
            response = al.RequestURLAndSave(url, False, config_file, analysis_id, doc_prefix, doc_id)
            soup = BeautifulSoup(response, 'lxml')
            links = []
            for container in soup.select('.b_algo h2 a'):
                actURL = BingToActualURL(container['href']) # need to find the actual url
                if actURL is not None:
                    links.append(actURL)
            links = links[:no_links]

            return links
    except Exception as e:
        logging.exception("error running bing search and save results")
        logging.exception(traceback.format_exc())
        raise Exception(e) 

l = SearchBingGetLinksAndSave(['top 3 factors that cause the price of nymex crude oil to increase'], 3, 'prod',analysis_id,'incFact')

In [42]:
l

['https://www.forbes.com/sites/forbesbooksauthors/2021/01/25/factors-that-influence-pricing-of-oil-and-gas/#:~:text=Weather%2C%20political%20disturbances%2C%20supply%20problems%20%E2%80%94%20these%20factors,supply%20and%20demand%20levels%2C%20prices%20will%20level%20out.',
 'https://www.cmegroup.com/openmarkets/energy/2022/Crude-Oil-Cause-or-Effect-of-Inflation.html',
 'https://www.cmegroup.com/openmarkets/energy/2022/Crude-Oil-Cause-or-Effect-of-Inflation.html']

In [16]:

    
print(response)

var u = "https://www.eia.gov/todayinenergy/detail.php?id=50738#:~:text=Crude%20oil%20prices%20increased%20in%202021%20as%20increasing,global%20petroleum%20demand%20rising%20faster%20than%20petroleum%20supply.";
        if (s)
          window.location.href = u;
        else
          window.location.replace(u);
      }
      function f() {
        document.getElementById("fb").style.display = "block";
      }
      //]]>
    </script>
  </head>
  <body onload="l()">
    <div id="fb" style="display: none">
      Please <a href="https://www.bing.com/ck/a?!&&p=e8b03e386126d51fJmltdHM9MTcwMDQzODQwMCZpZ3VpZD0wNTUxOWE1OS00ZGFjLTY4YmUtMmE4My04OTk2NGNkMzY5NzImaW5zaWQ9NTQ4Mg&ptn=3&ver=2&hsh=3&fclid=05519a59-4dac-68be-2a83-89964cd36972&u=a1aHR0cHM6Ly93d3cuZWlhLmdvdi90b2RheWluZW5lcmd5L2RldGFpbC5waHA_aWQ9NTA3MzgjOn46dGV4dD1DcnVkZSUyMG9pbCUyMHByaWNlcyUyMGluY3JlYXNlZCUyMGluJTIwMjAyMSUyMGFzJTIwaW5jcmVhc2luZyxnbG9iYWwlMjBwZXRyb2xldW0lMjBkZW1hbmQlMjByaXNpbmclMjBmYXN0ZXIlMjB0aGFuJTIwcGV0cm9sZXVtJTIwc3Vw

In [29]:
res2 = AnalyseEffect(subject, 'strong economy and greater demand', model_props, 'prod', analysis_id)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
from_string grammar:



root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number_28 
number_27 ::= [-

In [30]:
res2

{'query': 'how does strong economy and greater demand lead to an increase in the price of nymex crude oil',
 'result': '{ "the demand for all industrial commodities instead of the demand for all goods and services responds to both oil supply shocks and the aggregate demand shocks contemporaneously and it is influenced by the oil-market specific demand shocks with lags." \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 'source_documents': [Document(page_content='how strong economy and greater demand affects on the price of nymex crude oil - Search RewardsGaeilgeAllImagesVideosMapsNewsShoppingMoreFlightsTravelHotelsToolsAbout 25,400 re

In [6]:
res3 = json.loads(res2['result'])
res3

{'question': "How does OPEC's market and demand structure and demand elasticity affect the price of Nymex crude oil?",
 'answers': [{'text': "OPEC's market and demand structure and demand elasticity can cause an increase in the price of Nymex crude oil by influencing the global supply and demand balance, and by adjusting their production levels to meet changing market conditions.",
   'score': 80},
  {'text': "The price of Nymex crude oil is determined by factors such as global demand, supply, geopolitical events, and OPEC's actions have a limited impact on the price.",
   'score': 15},
  {'text': "OPEC's market and demand structure and demand elasticity can cause a decrease in the price of Nymex crude oil by increasing the global supply and reducing demand.",
   'score': 5}]}